### Providing necessary packages, URLs, and keys.
### Point to note: I haven't saved the API key to an environment file for temporary access and testing purposes.


In [53]:
import requests
import pandas as pd 
import datetime
seaching_city=''
base_url = 'https://api.openweathermap.org/data/2.5/weather'
forcast_url ='https://api.openweathermap.org/data/2.5/forecast'
api_key ="fa92cc369f0e56f67f813a44ede497cb" 

### We prompt the user to input their desired city. Using geocoding in conjunction with OpenWeatherMap, we retrieve the corresponding coordinates

In [54]:
def get_coordinate(city_name, api_key):
    """
    Getting latitude and longitude of a given city with openweather geocode api 
    """
    search_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name}&appid={api_key}"
    coordinate_response = requests.get(search_url)
    [result] =coordinate_response.json()
    return {'lat' : result['lat']  , 'lon':result['lon']}

### i fetched weather data and forecast data using their respective URLs and parameters, processing them before returning the results.

In [55]:
def get_weather_data(lat , lon, api_key):
    """ 
      Getting weather data base on provided lattiude and longitude 
    """
    parameters ={
        "lat":lat,
        'lon':lon,
        'appid':api_key
    }
    weather_data_response = requests.get(base_url , params=parameters)
    weather_forcast_response = requests.get(forcast_url , params=parameters)
    weather_data = weather_data_response.json()
    forcast_data = weather_forcast_response.json()

    return {"weather_data":weather_data ,"forcast_data": forcast_data}


### Using the weather API, we have the option to retrieve current weather data or forecast data, providing a list of 3-hour forecasts for the next 5 days.
### Within the 'clean_data' function, we have implemented an 'extract_value' method to effectively handle dictionaries and lists, extracting and cleaning their values.

In [56]:
def clean_data(weather_data, forcast_data):
    """
    retreiving what we want from result 
    """
    forcast_list =[]

    def extract_values(weather_dic): 
        temp_celsius = round(weather_dic['main']['temp'] - 273.15)
        weather_condition=weather_dic['weather'][0]['main']
        result ={
            'name': weather_dic.get('name' , forcast_data['city']['name']), # we ignored name for now 
            'condition':weather_condition, 
            'wind':weather_dic['wind']['speed'], 
            'temperature':temp_celsius ,
            'humidity':weather_dic['main']['humidity']
            }
        return result 
    # we got one object for weather and list of objects in forcast , so we extract it one by one and appended it in forcast_list
    for  weather_dic in forcast_data['list'] : 
        forcast_list.append(extract_values(weather_dic))
    
    weather_result = extract_values(weather_data)

    return {'weather_result':weather_result , 'forcast_list':forcast_list}


In [57]:
def main():
    """ 
    getting final data in json format
    """
    city_name = input('Enter city name: ').strip()
    coordinates = get_coordinate(city_name, api_key)
    data =get_weather_data(coordinates['lat'] , coordinates['lon'], api_key)
    final_data = clean_data(data['weather_data'] ,data['forcast_data'])
    return final_data , city_name


### After retrieving data in the main function, i structured two data frames one for today's weather and another for the forecast. 
### Key considerations: While i could have solely extracted today's weather from the first index of the forecast data, to ensure better list and object management , i opted to present both separately.

In [58]:
data , city_name = main()
today_keys = data['weather_result'].keys() 
today_dataFrame = pd.DataFrame(data['weather_result'] ,columns=today_keys , index=[0])
forcast_dataFrame = pd.DataFrame(data['forcast_list'])

### Subsequently, we stored our data in  CSV  formats.


In [59]:
today_dataFrame.to_csv(f'{city_name}_today_weather.csv')
forcast_dataFrame.to_csv(f'{city_name}_forcast.csv')

### Additional considerations:
### - Implementing error handling using try-except blocks.
### - Expanding the dataset by including more data points or diverse cities in the Pandas DataFrame.